In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Zobrist hashing

In [2]:
# Pkg.add("Chess")
using Chess
using Random
using NBInclude

In [3]:
@nbinclude("AdvancedBoard.ipynb")

undomoveAdv! (generic function with 1 method)

This notebook allows the user to hash the `Board` object into a bitstring. This is used to save `Board` in the cache efficiently.

Literature to Zobrist Hashing: https://www.chessprogramming.org/Zobrist_Hashing

Zobrist hashing is a hashing method which converts a chess board into an integer. This allows us to save the board in the Cache more efficiently. 

To calculate the integer a hashing table is created containing the following information:
1. an int for each piece on each square (12 unique pieces * 64 squares in total)
1. an int for each castling right for each side (4 in total)
1. an int for the file on which an en passant can occure (8 in total)
1. an int to indicate that it is black's turn (1 in total)

For each 'information' (as specified above) the corresponding int gets XOR'ed to the boards hash. The board's hash starts at a value of 0. For example: A white rook is located on the `A1` square. This means that the int for a white rook on the `A1` square is taken from the hashing table and XOR'ed with the hash value of 0. This is done for each piece, castling right, en passant file and turn color always modifying the value gotten from the previous calculations. This will always deliver a unique integer for each board, which is reproducible and reversible. The XOR operator is chose to be able to reverse any operations done easily as applying the XOR operator twice will always return the same value.

((a ⊻ b) ⊻ b) = b

A hash can be easily calculated iteratively as only each change needs to be XOR'ed with the previous hash. Given a hash `h` and moving a white rook from `A1` to `A2` means to XOR the value of a white rook on `A1` and XORing the value of a white rook on `A2` with the previos hash h.

The uniqueness of a hash is limited to the length of the integer. Collisions can happen after calculating round about 65.000 boards when using a 32-bit Hash and about 4 billion (4.000.000.000) boards when a using 64-bit hash.  

## Creating the hashing table

Each unique chess `Piece` is assigned a position which will determine the position in the `pieces` Matrix below.

In [4]:
if @isdefined(indices) == false
    const indices = Dict(PIECE_WP  => 1, 
               PIECE_WR  => 2, 
               PIECE_WN  => 3, 
               PIECE_WB  => 4,
               PIECE_WQ  => 5,
               PIECE_WK  => 6,
               PIECE_BP  => 7, 
               PIECE_BR  => 8, 
               PIECE_BN  => 9, 
               PIECE_BB  => 10,
               PIECE_BQ  => 11,
               PIECE_BK  => 12)
end

Dict{Piece, Int64} with 12 entries:
  PIECE_BR => 8
  PIECE_BB => 10
  PIECE_WK => 6
  PIECE_WQ => 5
  PIECE_BN => 9
  PIECE_WB => 4
  PIECE_BK => 12
  PIECE_WP => 1
  PIECE_WN => 3
  PIECE_WR => 2
  PIECE_BQ => 11
  PIECE_BP => 7

The `ZobristHashing` struct contains 4 attributes:

1. The `pieces` attribute has a two dimensional Array of 64 bit Integers. The size of this array will be 64x12. Each Square on a chess board is assigned 12 Integers representing the 12 chess pieces.
1. The `castling_rights` attribute consists of an integer array of length 4. It contains an Integer for each castling right. (White queen and King side and black queen and king side)
1. The `en_passant` attribute consists of an integer array of length 8. It contains an Integer for each file on the chess board. The file on which an en_passant square is available will be notated here.
1. The `turn` attribute consists of a single integer containing whether black has to move.

In [5]:
struct ZobristHashing
    pieces::Array{UInt64, 2}
    castling_rights::Array{UInt64, 1}
    en_passant::Array{UInt64, 1}
    turn::UInt64
end

### Constructor: generate_zobrist_hashing()

The `generate_zobrist_hashing` function is the basic constructor of a `ZobristHashing` table and initializes the table after the constraints above.

Returns:

1. `zobristHashing` A hashing table with pseudo-random values

In [6]:
function generate_zobrist_hashing()::ZobristHashing
    pieces = [rand(UInt64) for _ in 1:12, _ in 1:64]
    castling_rights = [rand(UInt64) for _ in 1:4]
    en_passant = [rand(UInt64) for _ in 1:8]
    turn = rand(UInt64)
    return ZobristHashing(pieces, castling_rights, en_passant, turn)
end

generate_zobrist_hashing (generic function with 1 method)

Init zobrist hashing table

In [7]:
if !@isdefined(zobrist)
    const zobrist = generate_zobrist_hashing()
end

ZobristHashing(UInt64[0x9ee0f751d6e2162e 0x0af468ba456be20e … 0xb472403ec5ee8aa5 0x286cb5ce11483b26; 0x7465b705b518e55e 0x4d782fca172ef39b … 0xc6d5cb8cb8c8c4cb 0x7d62b612f38b51b2; … ; 0x394db8351a98f15b 0x27820d271ef49560 … 0xc98a097a30b1834e 0x6b181efc06988209; 0x028de3d7f1ac5b00 0x73c2eee568b47cea … 0x387a354da2ee15e1 0xd82f00aeec07e451], UInt64[0xae789fe22a5382a8, 0xd715ef1af28ce106, 0x7a79932d6ff6e9f2, 0xbd1e7ae5e560ed22], UInt64[0x98d495793df25ec2, 0x579304c3e9e60ec2, 0xa6213fbbb64ca907, 0x600b6a9dd8919475, 0xd510910ad54d398b, 0xda84097cae36807f, 0x47f8782e7f723e86, 0x147619c9d1f40355], 0xccceef5040b9cda1)

## Non Incremental Zobrist Hashing

### Function: zobrist_hash (non incremental)

The `zobrist_hash` function takes in a `board` and a `zobrist` hashing table and hashes the `board` into an integer using the hashing table. To convert the `board` into an int the function begins with 0 as the hash. It then iterates over each piece on the board and applies the xor operation to the value for the piece on it's square gotten from the table with the hash. After that the values for the castling rights, en passant and move turn are also xor-ed to the hash. This creates a unique integer for each board. At the end this hash is returned and can be stored in a cache.

Arguments:

1. `board::Board` The board that needs to be hashed

Returns:

1. `hash::UInt64` The hash of the given board

In [8]:
function zobrist_hash(board::Board)::UInt64
    hash = 0
    # hash pieces
    for x in 1:8,y in 1:8
        square = Square(SquareFile(x), SquareRank(y))
        piece = pieceon(board, square)
        if piece != EMPTY
            hash = xor(hash, zobrist.pieces[indices[piece], square.val])
        end
    end
    
    # hash castling rights
    if cancastlekingside(board, WHITE)
        hash = xor(hash, zobrist.castling_rights[1])
    end
    if cancastlequeenside(board, WHITE)
        hash = xor(hash, zobrist.castling_rights[2])
    end
    if cancastlekingside(board, BLACK)
        hash = xor(hash, zobrist.castling_rights[3])
    end
    if cancastlequeenside(board, BLACK)
        hash = xor(hash, zobrist.castling_rights[4])
    end
    
    # hash en passant
    if epsquare(board) != SQ_NONE
        epfile = file(epsquare(board))
        hash = xor(hash, zobrist.en_passant[Int(tochar(epfile))-Int('a') + 1])
    end
    
    # hash turn color
    if sidetomove(board) == BLACK
        hash = xor(hash, zobrist.turn)
    end
    return hash
end

zobrist_hash (generic function with 1 method)

## Incremental Zobrist Hashing

### Function: updateCastleRightsHash
The function `updateCastleRightsHash` is an auxiliary for the `zobrist_hash` function. It checks whether the move done will change any castling rights and modifies the hash accordingly. If no castling rights are modified by the move the hash will not be changed.

Arguments:

1. `board::Board` The current board
1. `hash::UInt64` A hash value after some calculations of the `zobrist_hash` function
1. `move::Move`   The move for which the hash needs to be calculated

Returns:

1. `hash::UInt64` A hash of a board after applying the castling rights to the hash

In [9]:
function updateCastleRightsHash(board::Board, hash::UInt64, move::Move)::UInt64
    if (cancastlekingside(board, WHITE))
        undoinfo = domove!(board, move)
        if ! cancastlekingside(board, WHITE)
            hash = xor(hash, zobrist.castling_rights[1])
        end
        undomove!(board, undoinfo)
    end

    if (cancastlequeenside(board, WHITE))
        undoinfo = domove!(board, move)
        if ! cancastlequeenside(board, WHITE)
            hash = xor(hash, zobrist.castling_rights[2])
        end
        undomove!(board, undoinfo)
    end

    if (cancastlekingside(board, BLACK))
        undoinfo = domove!(board, move)
        if ! cancastlekingside(board, BLACK)
            hash = xor(hash, zobrist.castling_rights[3])
        end
        undomove!(board, undoinfo)
    end
    
    if (cancastlequeenside(board, BLACK))
        undoinfo = domove!(board, move)
        if ! cancastlequeenside(board, BLACK)
            hash = xor(hash, zobrist.castling_rights[4])
        end
        undomove!(board, undoinfo)
    end
    return hash 
end

updateCastleRightsHash (generic function with 1 method)

### Function: updateEnPassantHash
The function `updateEnPassantHash` is an auxiliary for the `zobrist_hash` function. It checks whether the move done will create an en passant square and modifies the hash accordingly. If no en passant square is made by the move the hash will not be modified.

Arguments:

1. `board::Board` The current board
1. `hash::UInt64` A hash value after some calculations of the `zobrist_hash` function
1. `move::Move`   The move for which the hash needs to be calculated

Returns:

1. `hash::UInt64` A hash of a board after applying the en passant square to the hash

In [10]:
function updateEnPassantHash(board::Board, hash::UInt64, move::Move)::UInt64
    undoinfo = domove!(board, move)
    if epsquare(board) != SQ_NONE
        epfile = file(epsquare(board))
        hash = xor(hash, zobrist.en_passant[Int(tochar(epfile))-Int('a') + 1])
    end
    undomove!(board, undoinfo)
    return hash
end 

updateEnPassantHash (generic function with 1 method)

### Function: zobrist_hash
The function `zobrist_hash` returns the zobrist hash of the given board after doing the given move. It only calculates the differences made via the given move and does not iterate over the whole board. Therefore, the performance of this function compared to the `zobrist_hash(board)` function is much better.

Arguments:

1. `board::Board` The current board
1. `hash::UInt64` The hash of the current board
1. `move::Move`   The move for which the hash needs to be calculated

Returns:

1. `hash::UInt64` The hash of the board after doing the given move on the current board

In [11]:
function zobrist_hash(board::Board, hash::UInt64, move::Move)::UInt64
    toMove = to(move)
    fromMove = from(move)
    piece = pieceon(board, from(move))
    pieceType = ptype(piece)
    color = Chess.pcolor(piece)
    hash = xor(hash, zobrist.pieces[indices[piece], fromMove.val])
    hash = xor(hash, zobrist.pieces[indices[piece], toMove.val])
    #Capturing pieces
    capturePiece = pieceon(board, toMove)
    if capturePiece != EMPTY
        hash = xor(hash, zobrist.pieces[indices[capturePiece], toMove.val])
    end
    
    #Castling
    if pieceType == KING
        if cancastlekingside(board, WHITE) && toMove == SQ_G1
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_H1.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_F1.val])
        elseif cancastlequeenside(board, WHITE) && toMove == SQ_C1
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_A1.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_D1.val])
        elseif cancastlekingside(board, BLACK) && toMove == SQ_G8
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_H8.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_F8.val])
        elseif cancastlequeenside(board, BLACK) && toMove == SQ_C8
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_A8.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_D8.val])
        end
    end
    
    hash = updateCastleRightsHash(board,hash, move)

    #En passant
    enpassantSquare = epsquare(board)
    if enpassantSquare != SQ_NONE 
        if enpassantSquare == toMove
            lastToMove = to(lastmove(board))
            if color ==WHITE 
                hash = xor(hash, zobrist.pieces[indices[PIECE_WP], lastToMove.val])
            else
                hash = xor(hash, zobrist.pieces[indices[PIECE_BP], lastToMove.val])
            end
        end
        epfile = file(epsquare(board))
        hash = xor(hash, zobrist.en_passant[Int(tochar(epfile))-Int('a') + 1])
    end

    hash = updateEnPassantHash(board,hash, move)
    #Promotion
    if ispromotion(move) 
        hash = xor(hash, zobrist.pieces[indices[piece], toMove.val])
        promoPiece = Piece(color,promotion(move))
        hash = xor(hash, zobrist.pieces[indices[promoPiece], toMove.val])
    end
    
    #Turn
    hash = xor(hash, zobrist.turn)
end

zobrist_hash (generic function with 2 methods)

In [12]:
function zobrist_hash_Adv(aBoard::AdvBoard, move::Move)::UInt64
    hash = aBoard.hash
    board = aBoard.board
    toMove = to(move)
    fromMove = from(move)
    display(board)
    print(fromMove)
    piece = pieceon(board, fromMove)
    pieceType = ptype(piece)
    color = Chess.pcolor(piece)
    println(piece)
    println(fromMove.val)
    println(indices[piece])
    hash = xor(hash, zobrist.pieces[indices[piece], fromMove.val])
    hash = xor(hash, zobrist.pieces[indices[piece], toMove.val])
    #Capturing pieces
    capturePiece = pieceon(board, toMove)
    if capturePiece != EMPTY
        hash = xor(hash, zobrist.pieces[indices[capturePiece], toMove.val])
    end
    
    #Castling
    if pieceType == KING
        if cancastlekingside(board, WHITE) && toMove == SQ_G1
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_H1.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_F1.val])
        elseif cancastlequeenside(board, WHITE) && toMove == SQ_C1
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_A1.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_WR], SQ_D1.val])
        elseif cancastlekingside(board, BLACK) && toMove == SQ_G8
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_H8.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_F8.val])
        elseif cancastlequeenside(board, BLACK) && toMove == SQ_C8
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_A8.val])
            hash = xor(hash, zobrist.pieces[indices[PIECE_BR], SQ_D8.val])
        end
    end
    
    hash = updateCastleRightsHash(board,hash, move)

    #En passant
    enpassantSquare = epsquare(board)
    if enpassantSquare != SQ_NONE 
        if enpassantSquare == toMove
            lastToMove = to(lastmove(board))
            if color == WHITE 
                hash = xor(hash, zobrist.pieces[indices[PIECE_WP], lastToMove.val])
            else
                hash = xor(hash, zobrist.pieces[indices[PIECE_BP], lastToMove.val])
            end
        end
        epfile = file(epsquare(board))
        hash = xor(hash, zobrist.en_passant[Int(tochar(epfile))-Int('a') + 1])
    end

    hash = updateEnPassantHash(board,hash, move)
    #Promotion
    if ispromotion(move) 
        hash = xor(hash, zobrist.pieces[indices[piece], toMove.val])
        promoPiece = Piece(color,promotion(move))
        hash = xor(hash, zobrist.pieces[indices[promoPiece], toMove.val])
    end
    
    #Turn
    hash = xor(hash, zobrist.turn)
end

zobrist_hash_Adv (generic function with 1 method)